In [119]:
from PIL import Image
import os, cv2, random
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
from time import time
from glob import glob
import keras
import random
import shutil

In [120]:
np.random.seed(999)

# Convert raw txt file to csv file where image index is from 1

In [27]:
df = pd.read_csv("PHI data/multi_labels.txt", delimiter='	',header=None, names=['path', 'task1', 'task2', 'task3', 'task4', 'task5', 'task6', 'task7', 'task8'], )

In [28]:
df.index = np.arange(1,len(df)+1)

In [29]:
df.head()

,path,task1,task2,task3,task4,task5,task6,task7,task8
1,task1/O/123.jpg,0,1,-1,1,-1,1,-1,-1
2,task1/O/1257.jpg,0,0,0,-1,-1,3,0,0
3,task1/O/1583.jpg,0,0,0,-1,-1,3,-1,-1
4,task1/O/1590.jpg,0,0,0,1,-1,3,-1,-1
5,task1/O/1592.jpg,0,0,0,0,-1,2,-1,-1


In [32]:
df.to_csv("multi_labels.csv")

# Select index to seperate training and test data

In [121]:
def generate_index(df, set_size):
    set_list = []
    while len(set_list)<set_size:
        # Generate random index from 1 to last index
        random_ind = random.randint(1, len(df))
        if random_ind not in set_list:
            set_list.append(random_ind)
    
    return set_list

In [122]:
def generate_index_another_set(df, input_list):
    
    # generate index for another set
    # if input list is training, the generated list here is for test
    another_list = []
    
    # index from 1, 2 to 36413
    for index in range(1, len(df)+1):
        if index not in input_list:
            another_list.append(index)
    
    return another_list

In [123]:
def compute_num_valid_data(df, task_id, label_list):
    label_all = df[task_id][label_list]
    num_label_all = len(label_all)
    num_label_task = num_label_all - label_all.value_counts()[-1]
#     print(num_label_task)
    return num_label_task

In [124]:
# Compute ratio between training and test
def compute_split_ratio(df, task_id, training_list, test_list):
#     print(compute_num_valid_data(df, task_id, training_list)/compute_num_valid_data(df, task_id, test_list))
        return compute_num_valid_data(df, task_id, training_list)/compute_num_valid_data(df, task_id, test_list)

In [125]:
def generate_split_ratio(df, split_ratio):
    
    # Loop until we find split ratio
    done = False
    iter_num = 1
        
    while not done:
        training_size = split_ratio*len(df)
        test_size = (1-split_ratio)*len(df)
#         print("training_size: ", training_size)
#         print("test_size: ", test_size)

#         training_list = generate_index(training_size)
        # sampling test index will cost less than generating training list
        test_list = generate_index(df, test_size)
        training_list = generate_index_another_set(df, test_list)
        print("Training size:", len(training_list), " Test size:", len(test_list))
#         print("Sampling index is done!")

        ratio_list = []

        for i in range(1,9):
            task_id = 'task' + str(i)
            ratio_list.append(compute_split_ratio(df, task_id, training_list, test_list))
        print("Iter", iter_num, ": ", ratio_list)
        
        # Make sure at least 8:1 but no larger than 9:1
        if (min(ratio_list) > 8) & (max(ratio_list) < 9):
            done = True
        
        iter_num += 1
    
    return training_list, test_list

In [130]:
df = pd.read_csv("multi_labels.csv", index_col=0)
df.head()

,path,task1,task2,task3,task4,task5,task6,task7,task8
1,task1/O/123.jpg,0,1,-1,1,-1,1,-1,-1
2,task1/O/1257.jpg,0,0,0,-1,-1,3,0,0
3,task1/O/1583.jpg,0,0,0,-1,-1,3,-1,-1
4,task1/O/1590.jpg,0,0,0,1,-1,3,-1,-1
5,task1/O/1592.jpg,0,0,0,0,-1,2,-1,-1


In [127]:
training_index_list, test_index_list = generate_split_ratio(df, 0.89)

Training size: 32407  Test size: 4006
Iter 1 :  [8.092907092907092, 8.465763195435093, 8.164691943127963, 7.642790697674418, 8.594405594405595, 7.83112582781457, 7.83047619047619, 7.851351351351352]
Training size: 32407  Test size: 4006
Iter 2 :  [8.153871940998995, 7.865063460253841, 7.61358574610245, 7.968146718146718, 8.333333333333334, 7.1187214611872145, 6.993103448275862, 6.960069444444445]
Training size: 32407  Test size: 4006
Iter 3 :  [8.038728897715988, 8.058703071672355, 8.241338112305854, 8.037937743190662, 8.27027027027027, 7.7730263157894735, 7.847328244274809, 7.851351351351352]
Training size: 32407  Test size: 4006
Iter 4 :  [8.153871940998995, 7.942722371967655, 8.230310262529834, 8.25398406374502, 7.265060240963855, 7.964705882352941, 8.072407045009784, 8.061264822134387]
Training size: 32407  Test size: 4006
Iter 5 :  [8.11111111111111, 8.08972602739726, 8.241338112305854, 8.490296220633299, 8.397260273972602, 8.212435233160623, 8.309236947791165, 8.31910569105691]


In [128]:
df_training_index = pd.DataFrame(data={"training_index": training_index_list})
df_test_index = pd.DataFrame(data={"test_index": test_index_list})

In [129]:
df_training_index.to_csv("training_index.csv", index=False)
df_test_index.to_csv("test_index.csv", index=False)

# Copy data into training and test folders

## Read and form dataframe for train and test index

In [4]:
df = pd.read_csv("multi_labels.csv", index_col=0)
df.head()

,path,task1,task2,task3,task4,task5,task6,task7,task8
1,task1/O/123.jpg,0,1,-1,1,-1,1,-1,-1
2,task1/O/1257.jpg,0,0,0,-1,-1,3,0,0
3,task1/O/1583.jpg,0,0,0,-1,-1,3,-1,-1
4,task1/O/1590.jpg,0,0,0,1,-1,3,-1,-1
5,task1/O/1592.jpg,0,0,0,0,-1,2,-1,-1


In [131]:
df_training_index = pd.read_csv("training_index.csv")
df_test_index = pd.read_csv("test_index.csv")

In [132]:
df_test_index_list = df_test_index['test_index'].tolist()
df_training_index_list = df_training_index['training_index'].tolist()

In [133]:
# Generate training index for all tasks
# loc is located by index, do not use iloc here
df_training = df.loc[df_training_index_list]
df_training.head()

,path,task1,task2,task3,task4,task5,task6,task7,task8
1,task1/O/123.jpg,0,1,-1,1,-1,1,-1,-1
3,task1/O/1583.jpg,0,0,0,-1,-1,3,-1,-1
4,task1/O/1590.jpg,0,0,0,1,-1,3,-1,-1
5,task1/O/1592.jpg,0,0,0,0,-1,2,-1,-1
6,task1/O/1603.jpg,0,0,0,-1,-1,1,-1,0


In [134]:
# Generate test index for all tasks
df_test = df.loc[df_test_index_list]
df_test.head()

,path,task1,task2,task3,task4,task5,task6,task7,task8
14650,task1/P/ds_7_p_00308.jpg,1,-1,-1,-1,-1,-1,-1,-1
6440,task1/O/ds_8_o_00833.jpg,0,-1,-1,-1,-1,-1,-1,-1
25709,task1/S/s_d00566.jpg,2,0,0,-1,0,-1,-1,-1
21534,task1/S/ds_9_s_00122.jpg,2,-1,-1,-1,-1,-1,-1,-1
24850,task1/S/n04311004_4123.JPEG,2,-1,-1,-1,-1,-1,-1,-1


In [135]:
# Find images have one specifed label
def find_img_loc_index(set_df, task_name, label):
    loc_index_list = set_df[set_df[task_name]==label][task_name].index.tolist()
    loc_index_list.sort()
    return loc_index_list

## Example of finding all images with label 1 in task2

In [10]:
df_training.loc[find_img_loc_index(df_training, 'task2', 1)].head()

,path,task1,task2,task3,task4,task5,task6,task7,task8
1,task1/O/123.jpg,0,1,-1,1,-1,1,-1,-1
9,task1/O/49.jpg,0,1,-1,0,-1,1,-1,-1
31,task1/O/ds_1_o_00019.jpg,0,1,-1,-1,-1,2,3,2
42,task1/O/ds_1_o_00030.jpg,0,1,-1,-1,-1,1,3,2
61,task1/O/ds_1_o_00049.jpg,0,1,-1,-1,-1,0,3,2


In [11]:
df.loc[find_img_loc_index(df_training, 'task2', 1)].head()

,path,task1,task2,task3,task4,task5,task6,task7,task8
1,task1/O/123.jpg,0,1,-1,1,-1,1,-1,-1
9,task1/O/49.jpg,0,1,-1,0,-1,1,-1,-1
31,task1/O/ds_1_o_00019.jpg,0,1,-1,-1,-1,2,3,2
42,task1/O/ds_1_o_00030.jpg,0,1,-1,-1,-1,1,3,2
61,task1/O/ds_1_o_00049.jpg,0,1,-1,-1,-1,0,3,2


In [136]:
task_label_registration = {'task1': ['O', 'P', 'S'],     # O:0; P:1; S:2
                           'task2' : ['D', 'UD'],       # Damage:0; Undamage:1
                           'task3' : ['NONSP', 'SP'],       # NONSP: 0; SP: 1 
                           'task4' : ['OTHER', 'STEEL'],       # OTHER: 0; STEEL: 1
                           'task5' : ['GC', 'NONC', 'PC'],    # GC: 0; NONC: 1; PC: 2
                           'task6' : ['BEAM', 'COL', 'ELSE', 'WALL'], # BEAM: 0; COL: 1; ELSE: 2; WALL: 3
                           'task7' : ['HEAVY', 'MIN', 'MOD', 'NO'], # HEAVY: 0; MIN: 1; MOD: 2; NO: 3
                           'task8' : ['COMB', 'FLEX', 'NO', 'SHEAR']} # COMB: 0; FLEX: 1; NO: 2; SHEAR: 3 

In [137]:
def find_all_img_paths(set_df, task_name, label):
    # find index of img for specific task and label
    # Eg. training_set, task2, Undamaged: label=1 
    loc_index_list = find_img_loc_index(set_df, task_name, label)
    # generate a list of path for such set (training or test)
    paths = set_df.loc[loc_index_list]['path'].tolist()
    return paths

In [138]:
def get_img_name(path_name):
    new_name = path_name.split('/')[-1]
    return new_name

In [149]:
# set_name "train" or "test"
def copy_img(set_name, task_name, label):
    print("Set name:", set_name, " Task name:", task_name, " Numeric label:", label, "Label", task_label_registration[task_name][label])
#     print(" ")
    
    if set_name == "train":
        df = df_training
    else:
        df = df_test
    
    # Find all image path in training or test
    all_img_names = find_all_img_paths(df, task_name, label)
    print("Total number of images: ", len(all_img_names))
    
    # Because some images may have same name but under different source folder.
    # E.g., S/1005.jpg and UD/1005.jpg
    duplicate_check_list = []
    for img in all_img_names:
    #     print(img)
        source_path = "PHI data/" + img
        source_name = get_img_name(img)
        # If find duplicate name of image, we add dup in front of target new name
        if source_name in duplicate_check_list:
            target_name =  "dup_"+ source_name
        else:
            target_name = source_name
        duplicate_check_list.append(source_name)
        
        target_path = "splited_data/" + task_name + "/" +  set_name + "/" + task_label_registration[task_name][label] + "/" + target_name
#         print(source_path)
#         print(target_path)
        shutil.copyfile(source_path, target_path)
    
    print("Done!")
    print(" ")

##  Example of copying images to train and test of task 8

In [62]:
copy_img("train", "task8", 0)
copy_img("train", "task8", 1)
copy_img("train", "task8", 2)
copy_img("train", "task8", 3)

Set name:  train  Task name:  task8  Label:  0
Total number of images:  1189
Done!
 
Set name:  train  Task name:  task8  Label:  1
Total number of images:  458
Done!
 
Set name:  train  Task name:  task8  Label:  2
Total number of images:  1633
Done!
 
Set name:  train  Task name:  task8  Label:  3
Total number of images:  830
Done!
 


In [63]:
copy_img("test", "task8", 0)
copy_img("test", "task8", 1)
copy_img("test", "task8", 2)
copy_img("test", "task8", 3)

Set name:  test  Task name:  task8  Label:  0
Total number of images:  121
Done!
 
Set name:  test  Task name:  task8  Label:  1
Total number of images:  64
Done!
 
Set name:  test  Task name:  task8  Label:  2
Total number of images:  175
Done!
 
Set name:  test  Task name:  task8  Label:  3
Total number of images:  112
Done!
 


# Generate all folders automatically

In [148]:
# Create splited_data folder
if not os.path.exists("splited_data"):
    os.makedirs("splited_data")
    print("Create folder: splited_data")

for i in range(1, 9):
    task_name = "task" + str(i)
    first_level_path = "splited_data/"+task_name
    # Create first level folder. E.g., splited_data/task1/
    if not os.path.exists(first_level_path):
        os.makedirsfirst_level_path
        print("Create folder:", first_level_path)

    # Create second level folder. E.g., splited_data/task1/train
    for set_name in ["train", "test"]:
        second_level_path = first_level_path+ "/" + set_name
        if not os.path.exists(second_level_path):
            os.makedirs(second_level_path)
            print("Create folder:", second_level_path)
        
        # Create third level folder. E.g., splited_data/task1/train/O
        num_labels = len(task_label_registration[task_name])
        for numeric_label in range(len(task_label_registration[task_name])):
            label = task_label_registration[task_name][numeric_label]
            third_level_path = second_level_path + "/" + label
            if not os.path.exists(third_level_path):
                os.makedirs(third_level_path)
                print("Create folder:", third_level_path)

Create folder: splited_data/task1/train/O
Create folder: splited_data/task1/train/P
Create folder: splited_data/task1/train/S
Create folder: splited_data/task1/test
Create folder: splited_data/task1/test/O
Create folder: splited_data/task1/test/P
Create folder: splited_data/task1/test/S
Create folder: splited_data/task2/train
Create folder: splited_data/task2/train/D
Create folder: splited_data/task2/train/UD
Create folder: splited_data/task2/test
Create folder: splited_data/task2/test/D
Create folder: splited_data/task2/test/UD
Create folder: splited_data/task3/train
Create folder: splited_data/task3/train/NONSP
Create folder: splited_data/task3/train/SP
Create folder: splited_data/task3/test
Create folder: splited_data/task3/test/NONSP
Create folder: splited_data/task3/test/SP
Create folder: splited_data/task4/train
Create folder: splited_data/task4/train/OTHER
Create folder: splited_data/task4/train/STEEL
Create folder: splited_data/task4/test
Create folder: splited_data/task4/test/

In [150]:
for i in range(1, 9):
    task_name = "task" + str(i)
    first_level_path = "splited_data/"+task_name

    # Create second level folder. E.g., splited_data/task1/train
    for set_name in ["train", "test"]:
        second_level_path = first_level_path+ "/" + set_name
        
        # Create third level folder. E.g., splited_data/task1/train/O
        num_labels = len(task_label_registration[task_name])
        for numeric_label in range(len(task_label_registration[task_name])):
            label = task_label_registration[task_name][numeric_label]
            third_level_path = second_level_path + "/" + label
            copy_img(set_name, task_name, numeric_label)

Set name: train  Task name: task1  Numeric label: 0 Label O
Total number of images:  8111
Done!
 
Set name: train  Task name: task1  Numeric label: 1 Label P
Total number of images:  7690
Done!
 
Set name: train  Task name: task1  Numeric label: 2 Label S
Total number of images:  8508
Done!
 
Set name: test  Task name: task1  Numeric label: 0 Label O
Total number of images:  962
Done!
 
Set name: test  Task name: task1  Numeric label: 1 Label P
Total number of images:  965
Done!
 
Set name: test  Task name: task1  Numeric label: 2 Label S
Total number of images:  1070
Done!
 
Set name: train  Task name: task2  Numeric label: 0 Label D
Total number of images:  6282
Done!
 
Set name: train  Task name: task2  Numeric label: 1 Label UD
Total number of images:  5529
Done!
 
Set name: test  Task name: task2  Numeric label: 0 Label D
Total number of images:  745
Done!
 
Set name: test  Task name: task2  Numeric label: 1 Label UD
Total number of images:  715
Done!
 
Set name: train  Task name:

# Statistic of labels

In [151]:
def count_img_num(set_name, task_name, label):
    
    if set_name == "train":
        df = df_training
    else:
        df = df_test
    
    
    print("Set name:", set_name, " Task name:", task_name, " Label:", label)
    
    num_img_by_df = len(find_all_img_paths(df, task_name, label))
    print("Num of images searched by dataframe: ", num_img_by_df)
    
    
    # Load original data path
    data_path ="splited_data/" + task_name + "/" + set_name + '/' + task_label_registration[task_name][label]

    # Store all image names in one list
    all_img_names = [x for x in os.listdir(data_path)]
    print("Num of images split by script: ", len(all_img_names))
    print(" ")

In [152]:
for i in range(1,9):
    task_name = "task" + str(i)
    num_of_labels = len(task_label_registration[task_name])
    
    for set_name in ['train', 'test']:
        for label in range(num_of_labels):
            count_img_num(set_name, task_name, label)

Set name: train  Task name: task1  Label: 0
Num of images searched by dataframe:  8111
Num of images split by script:  8111
 
Set name: train  Task name: task1  Label: 1
Num of images searched by dataframe:  7690
Num of images split by script:  7690
 
Set name: train  Task name: task1  Label: 2
Num of images searched by dataframe:  8508
Num of images split by script:  8508
 
Set name: test  Task name: task1  Label: 0
Num of images searched by dataframe:  962
Num of images split by script:  962
 
Set name: test  Task name: task1  Label: 1
Num of images searched by dataframe:  965
Num of images split by script:  965
 
Set name: test  Task name: task1  Label: 2
Num of images searched by dataframe:  1070
Num of images split by script:  1070
 
Set name: train  Task name: task2  Label: 0
Num of images searched by dataframe:  6282
Num of images split by script:  6282
 
Set name: train  Task name: task2  Label: 1
Num of images searched by dataframe:  5529
Num of images split by script:  5529
 

In [153]:
train_file_path_names = df_training['path'].tolist()
test_file_path_names = df_test['path'].tolist()

In [154]:
print(len(train_file_path_names))
print(len(test_file_path_names))

32407
4006


In [155]:
for img in train_file_path_names:
    duplicate_img_list = []
    if img in test_file_path_names:
        print(img)
    